<b> Import Library, find available devices </b>

In [35]:
# Initialization code for 1.8 V SiGe PMOS LDO Testing.
# This block identifies and initializes the connected SMUs (Source Measure Units).
# If running for the first time, ensure the SMU names are correctly referenced in the subsequent code block.

# The setup assumes the use of two SMUs:
# - One dedicated to VIN sweeps.
# - One Sinks a load current and measures VOUT.

import pyvisa                    # Instrument communication
import time                      # Handle time-related tasks (e.g., delays)
import matplotlib.pyplot as plt  # Plotting graphs and visualizing data
import numpy as np               # Numerical operations, particularly with arrays
import pandas as dp              # Data manipulation and analysis
import os                        # Interact with the operating system, such as handling file paths
import csv                       # Read from and write to CSV files
from datetime import date        # Read current date
from collections import defaultdict

In [ ]:
rm = pyvisa.ResourceManager()    # Create a ResourceManager object, which manages communication with instruments
rm.list_resources()              # List all available resources (e.g., connected instruments) managed by pyvisa

<b> Connect devices <b>

In [37]:
#Establishing Connection for VIN SMU
SM_VIN = rm.open_resource('GPIB6::20::INSTR')  # Open a connection to the instrument with the address 'ASRL3::INSTR' and assign it to SM_VIN
SM_VIN.read_termination = '\n'             # Set the read termination character to a newline. This tells pyvisa when to consider a message as complete when reading from the instrument.
SM_VIN.write_termination = '\n'            # Set the write termination character to a newline. This ensures that each command sent to the instrument is properly terminated.
SM_VIN.baud_rate = 9600                    # Set the baud rate for serial communication to 9600. This is the speed at which data is transmitted over the serial connection.
print(SM_VIN)                              # Print the resource object (SM_VIN) to display its properties
print(SM_VIN.query('*IDN?'))               # Send the '*IDN?' command to the instrument and print its response. This command usually asks the instrument to identify itself (e.g., manufacturer, model number).

#Establishing Connection for VOUT multimeter
MM_VOUT = rm.open_resource('USB0::10893::257::MY57508183::0::INSTR')
MM_VOUT.read_termination = '\n'             # Set the read termination character to a newline. This tells pyvisa when to consider a message as complete when reading from the instrument.
MM_VOUT.write_termination = '\n'
MM_VOUT.baud_rate = 9600
print(MM_VOUT)                             # Display resource's properties
print(MM_VOUT.query('*IDN?'))

#Establishing Connection for multimeter 1
MM_VDROP = rm.open_resource('USB0::0x2A8D::0x0101::MY54505624::INSTR')
MM_VDROP.read_termination = '\n'             # Set the read termination character to a newline. This tells pyvisa when to consider a message as complete when reading from the instrument.
MM_VDROP.write_termination = '\n'
MM_VDROP.baud_rate = 9600
print(MM_VDROP)                             # Display resource's properties
print(MM_VDROP.query('*IDN?'))              

#Establishing Connection for VREF waveform generator
WG = rm.open_resource('USB0::2391::22279::MY53802060::0::INSTR')
WG.read_termination = '\n'             # Set the read termination character to a newline. This tells pyvisa when to consider a message as complete when reading from the instrument.
WG.write_termination = '\n'
WG.baud_rate = 9600
print(WG)                             # Display resource's properties
print(WG.query('*IDN?'))

#Establishing Connection for Temperature Chamber
CHAMBER = rm.open_resource('GPIB6::6::INSTR') 
CHAMBER.read_termination = '\n'            
CHAMBER.write_termination = '\n'           
CHAMBER.baud_rate = 9600                  
print(CHAMBER)                            
print(CHAMBER.query('STATUS?'))              

GPIBInstrument at GPIB6::20::INSTR
KEITHLEY INSTRUMENTS INC.,MODEL 2401,4636506,B02 Jan 20 2021 10:19:49/B01  /W/N
USBInstrument at USB0::10893::257::MY57508183::0::INSTR
Keysight Technologies,34465A,MY57508183,A.02.17-02.40-02.17-00.52-04-01
USBInstrument at USB0::0x2A8D::0x0101::MY54505624::0::INSTR
Keysight Technologies,34465A,MY54505624,A.02.14-02.40-02.14-00.49-02-01
USBInstrument at USB0::2391::22279::MY53802060::0::INSTR
Agilent Technologies,33622A,MY53802060,A.01.11-2.25-03-64-02
GPIBInstrument at GPIB6::6::INSTR
YNNNYYNNNNNNNNNNNN0


<b> Define Constants <b>

In [33]:
Rhi=330 # ?
Rlo=68000 # ?

# Full VIN list created later. Minimum value used may vary.
VIN_MAX=1.8
VIN_STEP=.05
VIN_MIN=0 

ILMAX=-.01 # Never used. Also different sign from all other currents -Shawn
VIN=[]
ITOT=[]
VOUT=[]
IL_default=.003 # Load current to aim for

VREF_default = .6 # ? # This is used in load regulation tests and when tests are not actively being run

VREFlist = np.arange(.85, .3-.05, -.05) # From .85 V to .3 V in 50 mV steps
print(VREFlist)
TEMPlist = [125, 100, 75, 50, 25,  0, -25, -50, -75, -100, -125, -150, -175]
ILlist = [100E-6, 300E-6, 500E-6, 1E-3, 2E-3, 3E-3, 4E-3, 5E-3, 6E-3, 7E-3, 8E-3, 9E-3, 10E-3]
ILlistmA= [num * 1000 for num in ILlist]

[0.85 0.8  0.75 0.7  0.65 0.6  0.55 0.5  0.45 0.4  0.35 0.3 ]


<b> Initialize Source Meter (VIN) <b>

In [ ]:
#All of this is in SM_INIT, except for the print statement. Could use for testing. -Shawn
SM_VIN.write(":CONF:CURR")                  # Configure the SMU to measure current
SM_VIN.write("OUTP:STAT 0")                 # Turn off the VIN source meter
print(SM_VIN.query(":CONF?"))               # Query and print the current configuration of the SMU
SM_VIN.write("SOUR:VOLT:RANG 1.8")
SM_VIN.write("SENS:CURR:PROT:LEV 0.0105")   # Set the current protection level (compliance) to 10.5 mA. This limits the maximum current to protect the device under test.
SM_VIN.write("SENS:CURR:RANG 1E-2")         # Set the current measurement range to 10 mA. This sets the expected maximum current for accurate measurement.
SM_VIN.write("CURR:NPLC 10")                # Set the number of power line cycles (NPLC) for the current measurement to 10. 
                                            # This controls the integration time, with higher values leading to more accurate but slower measurements.

<b> Initialize Waveform Generator <b>

In [ ]:
#This is not in any function below. Might be necessary to run this. -Shawn
##### Waveform generator is used to output VREF #####
WG.write("OUTP1:LOAD DEF")          # Sets the output load to infinite so no impedance matching is necessary
WG.write("OUTP2:LOAD DEF")
WG.write("SOUR1:APPL:DC")          # Sets the Waveform generator to output a DC voltage
WG.write("SOUR2:APPL:DC")          # Sets the Waveform generator to output a DC voltage

<b> Initialize Multi Meter <b>

In [23]:
#All of this (and more) is in MMs_INIT, except for the print statements. Could use for testing. -Shawn
MM_VOUT.write(":CONF:VOLT:DC")
MM_VOUT.write("VOLT:DC:RANG 10")
print(MM_VOUT.query(":CONF?"))
print(MM_VOUT.query(":READ?"))

"VOLT +1.00000000E+01,+1.00000000E-06"
+1.98933964E+00


<b> Initialize Temperature Chamber <b>

In [ ]:
#All of this (and more) is in Chamber_INIT, even the print statements. Commented out for safety -Shawn
# CHAMBER.write("ON")
# CHAMBER.write("RATE=20")            # Sets the speed at which the chamber will ramp to change TEMP
# CHAMBER.write("WAIT=30")            # Sets the time that the chamber will wait once the desired temperature is reached before measurements start
# CHAMBER.write("HON")                # Turns heat on
# CHAMBER.write("CON")                # Turns cool on
# time.sleep(1) 

# print(CHAMBER.query("RATE?"))
# print(CHAMBER.query("WAIT?"))

<b> Initialization/On/Off Definitions </b>

In [30]:
def Chamber_INIT():
    CHAMBER.write("ON")
    CHAMBER.write("RATE=20")            # Sets the speed at which the chamber will ramp to change TEMP
    CHAMBER.write("WAIT=30")            # Sets the time that the chamber will wait once the desired temperature is reached before measurements start
    CHAMBER.write("HON")                # Turns heat on
    CHAMBER.write("CON")                # Turns cool on
    CHAMBER.write("PIDA=3")
    time.sleep(1) 

    print(CHAMBER.query("RATE?"))
    print(CHAMBER.query("WAIT?"))
    return

# This one is never called -Shawn
def SMU_On():
    SM_VIN.write(":SOUR:VOLT:MODE FIX")
    SM_VIN.write("SOUR:VOLT:LEV 0")
    SM_VIN.write("OUTP ON")
    return

def SMU_INIT():
    SM_VIN.write(":CONF:CURR")                  # Configure the SMU to measure current
    SM_VIN.write("OUTP:STAT 0")                 # Turn off the VIN source meter
    SM_VIN.write("SOUR:VOLT:RANG 1.8")
    SM_VIN.write("SENS:CURR:PROT:LEV 0.0105")   # Set the current protection level (compliance) to 10.5 mA. This limits the maximum current to protect the device under test.
    SM_VIN.write("SENS:CURR:RANG 1E-2")         # Set the current measurement range to 10 mA. This sets the expected maximum current for accurate measurement.
    SM_VIN.write("CURR:NPLC 10")
    return

def MMs_INIT():
    MM_VOUT.write(":CONF:VOLT:DC")
    MM_VOUT.write("VOLT:DC:RANG 10")
    MM_VOUT.write("VOLT:NPLC 10")
    MM_VDROP.write(":CONF:VOLT:DC")
    MM_VDROP.write("VOLT:DC:RANG 10")
    MM_VDROP.write("VOLT:NPLC 10")
    return

def turn_on():
    WG.write("SOUR1:APPL:DC DEF, DEF, " + str(VREF_default)) # Sets VREF to default value
    #TODO: TURN ON VREF BEFORE VIN? -Lang
    SM_VIN.write("SOUR:VOLT:LEV 1.8")                        # Set VIN source value
    SM_VIN.write("OUTP:STAT 1")                              # Turn on the VIN source meter output
    time.sleep(.5)
    VISET= -Rhi*(IL_default-VREF_default/Rlo) + VREF_default
    WG.write("SOUR2:APPL:DC DEF, DEF," + str(VISET))         # Turn on the WG for VISET to set IL to default for the first VREF so that it can warm up while CHAMBER starts
    return

def turn_off():
    WG.write("SOUR2:APPL:DC DEF, DEF, 0")                    # Turn off VISET first!!!
    time.sleep(.5)
    SM_VIN.write("OUTP:STAT 0")                              # Turn off the VIN source meter
    WG.write("OUTP1:STAT 0")                                 # Turn off VREF
    return

<b> Test Sweep Definitions </b>

In [31]:
#Varies VREF and, internally, VIN. 
#Records VOUTs for Line Regulation Plots
#Records the VOUT corresponding to the largest VIN for the Voltage Regulation Plot
#Records calculated Dropout Voltage (from smallest VIN and corresponding VOUT) for the Dropout Voltage Plot
def VREF_Sweep(local, TEMP, LRplot):
    VISETlist=[]
    VDOlist=[]
    VOUTreglist=[]
    for VREF in VREFlist:                                   
        ####### SETS VREF #######
        VISET = -Rhi*(IL_default-VREF/Rlo) + VREF
        VISETlist.append(VISET)
        WG.write("SOUR2:APPL:DC DEF, DEF, " + str(VISET))   # Sets VISET to needed value to achieve default IL
        time.sleep(.1)                                      # Set VISET before VREF because VREF is being swept up. Want to limit current
        WG.write("SOUR1:APPL:DC DEF, DEF, " + str(VREF))    # Sets VREF

        VDO, VOUTreg, VIN, VOUT = VIN_Sweep(VREF, local)    # Calls the VIN_Sweep function that sweeps VIN
        
        LRplot[VREF].plot(VIN,VOUT, label = "T=" + str(TEMP) + " °C")
        VDOlist.append(VDO)
        VOUTreglist.append(VOUTreg)                                 
    # At the end of the sweep, restore defaults
    WG.write("SOUR1:APPL:DC DEF, DEF, " + str(VREF_default)) #VREF
    VISET = -Rhi*(IL_default-VREF_default/Rlo) + VREF_default #IL (with VISET)
    WG.write("SOUR2:APPL:DC DEF, DEF," + str(VISET))

    # Create an excel file with info on each VISET value used per VREF
    df2 = dp.DataFrame({'VREF': VREFlist, 'VISET': VISETlist})
    csv_path = os.path.join(local, "VISET") 
    df2.to_csv(csv_path, index=False)

    return(VDOlist, VOUTreglist)

#Called from VREF_Sweep, runs for each VREF value
def VIN_Sweep(VREF, local):
    ####### Creates Lists for Data #######
    SHEET="VIN_Sweep_VREF="+str(VREF)+".csv"                
    VIN=[]                                                  
    ITOT=[]                                                 # Stores total current values (IQ + IL)
    VOUT=[]                                                 
    VDROP=[]

    ####### Sweep of VIN for default IL #######
    # VIN starts high at 1.8 V and then sweeps down to VIN_MIN or VREF, whichever is higher.                                        
    # Note: IL should already be at -10mA (ILMAX) so no changes are needed to VIN SENS range or VOUT SOURCE value     #No clue what this means -Shawn
    new_MIN = max(VIN_MIN, VREF-.1)

    for val in np.arange(VIN_MAX, new_MIN, -VIN_STEP):      # Loop from VIN_MAX to VIN_MIN in steps of VIN_STEP                                                            
        val = round(val,2)                                     
        ####### Sets VIN voltage #######            
        SM_VIN.write("SOUR:VOLT:LEV " + str(val))           # Write the command to set the VIN voltage on the source meter
        time.sleep(.1)                                      # Pause for .5 seconds to allow the system to stabilize
        
        ####### Measure ILOAD and others #######
        VOUT.append(float(MM_VOUT.query(":READ?")))     
        ITOT.append(SM_VIN.query(":READ?").split(',')[1])   # Query and append the measured total current through VIN
        VDROP.append(float(MM_VDROP.query(":READ?")))
        VIN.append(val)                                     # Append VIN to list
    SM_VIN.write("SOUR:VOLT:LEV 1.8") 

    ####### Store Data #######
    df = dp.DataFrame({'VIN': VIN, 'ITOT': ITOT, 'VOUT': VOUT, 'VDROP': VDROP}) # Create a pandas DataFrame from VIN, ITOT, and VOUT lists
    csv_path = os.path.join(local, SHEET)                     # Generate the file path for saving the CSV
    df.to_csv(csv_path, index=False)                        # Save the DataFrame to a CSV file without including the index
    
    VDO = VIN[-1] - VOUT[-1]                                    # Calculate Dropout Voltage, to be used to generate plot.
    #VDO plot seems to be using the smallest VIN value (max of VIN_MIN, VREF-.1)
    
    return(VDO, VOUT[0], VIN, VOUT)


# Saves IL values and their Vout values for load regulation plot
def IL_Sweep(VREF, local, TEMP, LoadRplot):
    ####### Creates Lists for Data #######
    SHEET="IL_Sweep_VREF="+str(VREF)+".csv"                 
    ITOT=[]                                                 # Stores total current values (IQ + IL)
    VOUT=[]                                                
    VDROP=[]
    for IL in ILlist:
        VISET = -Rhi*(IL-VREF/Rlo) + VREF
        WG.write("SOUR2:APPL:DC DEF, DEF, "+str(VISET))
        time.sleep(.1)
        VOUT.append(float(MM_VOUT.query(":READ?")))     
        ITOT.append(SM_VIN.query(":READ?").split(',')[1])   # Not used for plots yet, saved anyways
        VDROP.append(float(MM_VDROP.query(":READ?")))       # Not used for plots yet, saved anyways
    VISET = -Rhi*(IL_default-VREF/Rlo) + VREF
    WG.write("SOUR2:APPL:DC DEF, DEF, " +str(VISET))

    ####### Store and Plot Data #######
    df = dp.DataFrame({'IL': ILlist, 'ITOT': ITOT, 'VOUT': VOUT, 'VDROP': VDROP})      # Create a pandas DataFrame from lists
    csv_path = os.path.join(local, SHEET)                      
    df.to_csv(csv_path, index=False)

    LoadRplot[VREF].plot(ILlistmA,VOUT, label="T=" + str(TEMP) + " °C")


<b> Plotting Helper</b>

In [34]:
def default_plot(title, xlabel, ylabel):
    # Create a color map for the plots
    cm=plt.get_cmap('gist_rainbow')

    # Apply any parameters that are shared by all plots
    fig, plot = plt.subplots(layout='constrained')
    plot.set_title(title, fontdict={'fontsize': 16, 'fontweight': 'bold'}, y = 1.03)
    plot.set_xlabel(xlabel, fontdict={'fontsize': 12})
    plot.set_ylabel(ylabel, fontdict={'fontsize': 12})
    plot.tick_params(axis='both', which='major', labelsize=10)
    plot.set_prop_cycle ('color', [cm(1.*i/len(TEMPlist)) for i in range(len(TEMPlist))])
    plot.grid()
    return fig, plot

<b> Test Protocol </b>

In [ ]:
def Run_Experiment(UseChamber):
    if (UseChamber):
        name = "/DUT_"
    else:
        name = "/Test_Run_"

    
    ################## Create Data Storage Directory ##################
    
    # Create directory
    DATE = "NPN_LVR_Results/"+str(date.today())     # Create a directory with the date.
    os.makedirs(DATE, exist_ok=True)                # If the directory already exists, it won't raise an error due to exist_ok=True
    i = 1
    DUT = str(DATE)+name+str(i)
    while os.path.isdir(DUT) == True and len(os.listdir(DUT)) != 0 : # Keeps track of the number of tests done today and checks if the folder is empty
        DUT = str(DATE)+name+str(i)                
        i = i+1
    os.makedirs(DUT, exist_ok=True)                 # Create another directory inside date directory for this trial
    
    
    ################## Initialize and format plots ##################
    
    # Dropout voltage plot
    DVfig, DVplot = default_plot('Dropout Voltage', 'Temperature (°C)', 'Vdo(Vin-Vout) (V)')
    #Vplot.set_yticks(np.arange(0, 1.3, 0.1))
    DVplot.set_xticks(np.arange(-150, 150, 50))
    DVplot.set_xlim(-175, 125)
    #DVplot.set_ylim(-175, 125)
    
    # Vout regulation plot
    VRfig, VRplot = default_plot('Vout Regulation at Vin=1.8V', 'Temperature (°C)', 'Vout (V)')
    VRplot.set_xticks(np.arange(-150, 150, 50))
    VRplot.set_xlim(-175, 125)
    
    LRplot = defaultdict(list)
    LRfig = defaultdict(list)
    # Create figures for the overall line regulation plots for each VREF
    for VREF in VREFlist:
        LRfig[VREF], LRplot[VREF] = default_plot('Line Regulation at VREF='+str(VREF)+'V', 'Vin (V)', 'Vout (V)')
        new_MIN = max(VIN_MIN, VREF-.1)
        LRplot[VREF].set_xticks(np.arange((new_MIN+.1), 1.8, 0.1))
        LRplot[VREF].set_xlim((new_MIN+.1), 1.8)
    
    # Load regulation plot
    VREF = VREF_default
    LoadRfig, LoadRplot = default_plot('Load Regulation at VREF='+str(VREF)+'V', 'IL (mA)', 'Vout (V)')
    LoadRplot.set_xticks(np.arange(-1, 11, 1))
    LoadRplot.set_xlim(0, ILlistmA[-1])
    #LoadRplot.set_xscale("log")
    
    
    ################## Initialize Equipment ##################
    
    if (UseChamber):
        Chamber_INIT()
    SMU_INIT()
    MMs_INIT()
    turn_on()
    
    
    ################## Temperature Sweep ##################
    
    VDOdict = defaultdict(list)
    VOUTregdict = defaultdict(list)
    
    if (UseChamber):
        TempsToTest = TEMPlist
    else:
        TempsToTest = [27]
        
    for TEMP in TempsToTest:
        local=str(DUT)+"/TEMP_"+str(TEMP)                           # The temperature of the experiment gets set as the sub directory name
        os.makedirs(local, exist_ok=True)                           # Create a directory with the given name. If the directory already exists, it won't raise an error due to exist_ok=True

        if (UseChamber):
            CHAMBER.write("WAIT=30")                                # Resets the wait time of the temp chamber to 30 minutes so it doesn't time out mid test
            CHAMBER.write("SET=" + str(TEMP))
            # TODO: There is probably a better way to check for temperature settling and do wait time
            while (CHAMBER.query("STATUS?")[3] == 'N'):             # Checks if the temperature chamber has arrived at TEMP yet or not
                time.sleep(15)
                print(CHAMBER.query("STATUS?"))  
            print(CHAMBER.query("STATUS?"))
            time.sleep(300)                                         # Soak time
    
        VDOlist, VOUTreglist = VREF_Sweep(local, TEMP, LRplot) # Performs every test, sweeping VREF
        IL_Sweep(VREF_default, local, TEMP, LoadRplot) #Performs IL_Sweep, which does not sweep VREF

        ####### Store data. Generate Intermediary line/load plots. #######
        for i, VREF in enumerate(VREFlist):
            VDOdict[VREF].append(VDOlist[i])
            VOUTregdict[VREF].append(VOUTreglist[i])
       
        for VREF in VREFlist:
            for line in LRplot[VREF].legend(bbox_to_anchor=(1.02, 1), loc='upper left').get_lines():
                line.set_linewidth(2.5)
            LRfig[VREF].savefig(os.path.join(local, "Line_Regulation_"+str(VREF)+"V.png"))
            for line in LoadRplot[VREF].legend(bbox_to_anchor=(1.02, 1), loc='upper left').get_lines():
                line.set_linewidth(2.5)
            LoadRfig[VREF].savefig(os.path.join(local, "Load_Regulation_"+str(VREF)+"V.png"))
    
    ################## Plot and Save ##################

    # Dropout voltage plot and Vout regulation plot
    for VREF in VREFlist:
        DVplot.plot(TempsToTest, VDOdict[VREF], label="VREF=" + str(VREF) + " (V)")
        VRplot.plot(TempsToTest, VOUTregdict[VREF], label="VREF=" + str(VREF) + " (V)")
        
    for line in DVplot.legend(bbox_to_anchor=(1.02, 1), loc='upper left').get_lines():
        line.set_linewidth(2.5)
    for line in VRplot.legend(bbox_to_anchor=(1.02, 1), loc='upper left').get_lines():
        line.set_linewidth(2.5)

    DVfig.savefig(os.path.join(DUT, "Dropout_Voltage.png"))
    VRfig.savefig(os.path.join(DUT, "Voltage_Regulation.png"))

    # Line regulation plots
    for VREF in VREFlist:
        for line in LRplot[VREF].legend(bbox_to_anchor=(1.02, 1), loc='upper left').get_lines():
            line.set_linewidth(2.5)
        LRfig[VREF].savefig(os.path.join(DUT, "Line_Regulation_"+str(VREF)+"V.png"))

    # Load regulation plot
    for line in LoadRplot[VREF].legend(bbox_to_anchor=(1.02, 1), loc='upper left').get_lines():
        line.set_linewidth(2.5)
    LoadRfig[VREF].savefig(os.path.join(DUT, "Load_Regulation_"+str(VREF)+"V.png"))
    
    
    ####################### Cold Start Test #######################
    
    # turn_off()                    # After temperature testing is complete, turn off the source meters
    # CHAMBER.write("WAIT=30")      # Reset wait time so that it doesn't time out
    # time.sleep(900)               # Wait 15 minutes for device to cool down
    # CHAMBER.write("WAIT=30")      # Same as above (TODO: There is probably a better way to do this)
    # turn_on()
    # time.sleep(300)               # Let the devices warm up again. Then run another measurement
    
    # CHAMBER.write("WAIT=30") 
    # local=str(DUT)+"/TEMP_-175C_Cold_Start"
    # os.makedirs(local, exist_ok=True)
    
    # VREF_Sweep(local, TEMP, LRplot, LoadRplot)
    
    
    ##################### Shut Down Procedure #####################
    
    turn_off()

    if (UseChamber):
        CHAMBER.write("SET=30")                             # Sets the temperature back to room temp once test is complete                  
        while (CHAMBER.query("STATUS?")[3] == 'N'):         # Checks if chamber has arrived at room temp
            time.sleep(15)
        CHAMBER.write("STOP")                               # Once chamber is at room temp, chamber stops operation


+20.000
00:30:00
YNNNYYYNYNNNNNNNNN0
YNNNYYYNYNNNNNNNNN0
YNNNYYYYYNNNNNNNNN0
YNNNYYYYYNNNNNNNNN0
YNNNYYYYYNNNNNNNNN0
YNNNYYYYYNNNNNNNNN0
YNNNYYYYYNNNNNNNNN0
YNNNYYYYYNNNNNNNNN0
YNNNYYYYYNNNNNNNNN0
YNNNYYYYYNNNNNNNNN0
YNNNYYYYYNNNNNNNNN0
YNNNYYYYYNNNNNNNNN0
YNNNYYYYYNNNNNNNNN0
YNNNYYYYYNNNNNNNNN0
YNNNYYYYYNNNNNNNNN0
YNNNYYYYYNNNNNNNNN0
YNNNYYYYYNNNNNNNNN0
YNNNYYYYYNNNNNNNNN0
YNNNYYYYYNNNNNNNNN0
YNNNYYYYNNNNNNNNNN0
YNNNYYYYNNNNNNNNNN0
YNNNYYYYNNNNNNNNNN0
YNNYYYYYNNNNNNNNNN0
YNNYYYYYNNNNNNNNNN0
YNNNYYYYYNNNNNNNNN0
YNNNYYYYYNNNNNNNNN0
YNNNYYYYYNNNNNNNNN0
YNNNYYYYYNNNNNNNNN0
YNNNYYYYNNNNNNNNNN0
YNNNYYYYNNNNNNNNNN0
YNNNYYYYNNNNNNNNNN0
YNNNYYYYNNNNNNNNNN0
YNNNYYYYNNNNNNNNNN0
YNNNYYYYNNNNNNNNNN0
YNNNYYYYNNNNNNNNNN0
YNNNYYYYNNNNNNNNNN0
YNNYYYYYNNNNNNNNNN0
YNNYYYYYNNNNNNNNNN0
YNNNYYYYYNNNNNNNNN0
YNNNYYYYYNNNNNNNNN0
YNNNYYYYYNNNNNNNNN0
YNNNYYYYYNNNNNNNNN0
YNNNYYYYNNNNNNNNNN0
YNNNYYYYNNNNNNNNNN0
YNNNYYYYNNNNNNNNNN0
YNNNYYYYNNNNNNNNNN0
YNNNYYYYNNNNNNNNNN0
YNNNYYYYNNNNNNNNNN0
YNNNYYYYNNNNNNNNNN0
YNN

In [ ]:
# TEST Run, chamber stays at room temperature
Run_Experiment(False)

In [ ]:
#
#
# Buffer so we don't misclick
#
#

In [ ]:
# ACTUAL Run, chamber sweeps temperature
Run_Experiment(True)